# 13.3 PolishNewsApp

Aplikacja wykorzystuje architekturę **MVVM**.

<img src="https://miro.medium.com/max/1100/0*PKo4mQsOOGUqPlVp.webp" width="600"/>

- Aplikacja będzie pokazywać listę 10 wiadomości z Polski
- Jako zewnętrzne źródło danych wykorzystamy https://newsdata.io (`Retrofit`, `LoggingInterceptor`)
- W aplikacji zastosujemy *offline caching* za pomocą `ROOM`, wraz z funkcją `networkBoundResource` oraz klasą `Resource`
- Zapisane dane będą czyszczone co 10 minut
- Aplikacja wykorzystuje strumienie (`Flow`, `StateFlow`) oraz kanały (`Channel`)
- Można dodać artykuły do listy ulubionych (`ROOM`)
- Aplikacja zawiera dwa fragmenty - nawigacja zostanie wykonana z pomocą bibliotek `BottomNavigation` oraz `Navigation`
- Wykorzystamy *dependency injection* z bibliotekami `Dagger-Hilt`
- Dane będą wyświetlane w `RecyclerView` z `ListAdapter`

<table><tr><td><img src="https://media3.giphy.com/media/uER0JwmDholYnLXChH/giphy.gif?cid=790b76119058b2b2f72a5a208fa580c5adabc5d007d55022&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media2.giphy.com/media/GprR3qLfjPdna6GY4m/giphy.gif?cid=790b76116f13b2b93dea78771b7ec3023cdf6f9877e4443a&rid=giphy.gif&ct=g" width="200" /></td>
<td><img src="https://media2.giphy.com/media/GmKwtKs1YnfnrQxEPG/giphy.gif?cid=790b761102b60ac194f6443efdb6ac545a2d184ce159195b&rid=giphy.gif&ct=g" width="200" /></td></tr></table>

### Konfiguracja

Rozpoczniemy od skonfigurowania projektu, dodania odpowiednich zależności oraz dodania pola z `apikey` do projektu. Rozpocznijmy od zależności.

In [ ]:
// build.gradle(Project)
buildscript { // przed blokiem plugins
    repositories {
        google()
    }
    dependencies {
        classpath "androidx.navigation:navigation-safe-args-gradle-plugin:2.5.3"
    }
}
plugins {
    id 'com.android.application' version '7.3.1' apply false
    id 'com.android.library' version '7.3.1' apply false
    id 'org.jetbrains.kotlin.android' version '1.7.20' apply false
    id 'com.google.dagger.hilt.android' version '2.44' apply false
}

In [ ]:
//  build.gradle(Module)
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'androidx.navigation.safeargs.kotlin'
    id 'kotlin-android'
    id 'kotlin-kapt'
    id 'dagger.hilt.android.plugin'
}
...
android {
    buildFeatures {
        viewBinding = true
    }
    ...
}
...
dependencies {

    // Dagger-Hilt
    implementation "com.google.dagger:hilt-android:2.44"
    kapt "com.google.dagger:hilt-android-compiler:2.44"
    kapt "androidx.hilt:hilt-compiler:1.0.0"
    implementation 'androidx.hilt:hilt-navigation-fragment:1.0.0'

    // Navigation
    implementation "androidx.navigation:navigation-fragment-ktx:2.5.3"
    implementation "androidx.navigation:navigation-ui-ktx:2.5.3"


    // Fragment
    implementation "androidx.fragment:fragment-ktx:1.5.4"

    // ROOM
    implementation("androidx.room:room-ktx:2.4.3")
    kapt("androidx.room:room-compiler:2.4.3")

    // ViewModel
    implementation 'androidx.lifecycle:lifecycle-viewmodel-ktx:2.5.1'
    // LiveData
    implementation "androidx.lifecycle:lifecycle-livedata-ktx:2.5.1"

    // Retrofit
    implementation 'com.squareup.retrofit2:retrofit:2.9.0'
    implementation 'com.squareup.retrofit2:converter-gson:2.9.0'

    // OkHttp
    implementation 'com.squareup.okhttp3:logging-interceptor:4.10.0'

    // Glide
    implementation 'com.github.bumptech.glide:glide:4.14.1'
    annotationProcessor 'com.github.bumptech.glide:compiler:4.14.1'

    // Paging3
    implementation "androidx.paging:paging-runtime-ktx:3.1.1"

    //SwipeRefreshLayout
    implementation "androidx.swiperefreshlayout:swiperefreshlayout:1.1.0"

    // CardView
    implementation "androidx.cardview:cardview:1.0.0"
    
    ...
}

Dodajmy odpowiednie upoważnienia

In [ ]:
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools">

    <uses-permission android:name="android.permission.INTERNET"/>
    <uses-permission android:name="android.permission.ACCESS_NETWORK_STATE"/>

Ponieważ https://newsdata.io/ wymaga posiadania unikalnego `apikey`, musimy go przechować w projekcie - jednocześnie nie chcemy go dodawać do repozytorium github. Sam klucz będziemy przechowywać w pliku `local.properties`

In [ ]:
NEWS_DATA_IO_KEY = "klucz"

Musimy utworzyć pole do którego możemy odnieść się w projekcie, zrobimy to przy pomocy `gradle`. W ppbloku `defaultConfig` (`build.gradle(Module)`) tworzymy instancję `Properties` i wywołujemy metodę `load` (załadowanie pliku) oraz `newDataInputStream` (tworzy strumień  wejścia).

In [ ]:
android {
    ...
    defaultConfig {
        ...

        Properties properties = new Properties()
        properties.load(project.rootProject.file("local.properties").newDataInputStream())
    }
    ...
}

Następnie wywołujemy metodę `buildConfigField` pozwalającą na utworzenie pola dostępnego z poziomu projektu przez klasę `BuildConfig`

In [ ]:
android {
    ...
    defaultConfig {
        ...

        Properties properties = new Properties()
        properties.load(project.rootProject.file("local.properties").newDataInputStream())
        
        buildConfigField(
            "String", // typ
            "NEWS_DATA_IO_KEY", // nazwa
            "\"${properties.getProperty("NEWS_DATA_IO_KEY")}\"" // lokalizacja
        )
    }
    ...
}

W projekcie możemy dostać `apikey` przez wywołanie

In [ ]:
const val API_KEY = BuildConfig.NEWS_DATA_IO_KEY

### Layout + Nawigacja

Dodajmy do nawigacji dwa fragmenty - każdy fragment dodajemy do odpowiedniego pakietu
- `LatestNewsFragment` - lista najnowszych wiadomości - pakiet: `ui.features.latest`
- `FavoriteNewsFragment` - lista ulubionych wiadomości - pakiet: `ui.features.favorite`
- aktywność dodajemy do pakietu `ui`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/latestNewsFragment">

    <fragment
        android:id="@+id/latestNewsFragment"
        android:name="pl.udu.uwr.pum.polishnewsapp.ui.features.latest.LatestNewsFragment"
        android:label="@string/wiadomo_ci"
        tools:layout="@layout/fragment_latest_news" />
    <fragment
        android:id="@+id/favoriteNewsFragment"
        android:name="pl.udu.uwr.pum.polishnewsapp.ui.features.favorite.FavoriteNewsFragment"
        android:label="@string/ulubione"
        tools:layout="@layout/fragment_favorite_news" />
</navigation>

Ponieważ będziemy wykorzystywać `BottomNavigation`, musimy dodać plik `menu` z elementami które chcemy tam umieścić.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@id/latestNewsFragment"
        android:icon="@drawable/ic_latest"
        android:title="@string/wiadomo_ci" />
    <item
        android:id="@id/favoriteNewsFragment"
        android:icon="@drawable/ic_favorite"
        android:title="@string/ulubione" />
</menu>

Dodajmy `FragmentContainerView` oraz `BottomNavigationView` do layoutu głównej aktywności

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".ui.MainActivity">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/nav_host_fragment"
        android:name="androidx.navigation.fragment.NavHostFragment"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        app:defaultNavHost="true"
        app:navGraph="@navigation/navigation"/>

    <com.google.android.material.bottomnavigation.BottomNavigationView
        android:id="@+id/bottom_nav_view"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        app:menu="@menu/bottom_nav_menu" />

</LinearLayout>

Przejdźmy do layoutu `LatestNewsFragment`. Wszystko umieścimy `CoordinationLayout`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.coordinatorlayout.widget.CoordinatorLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.features.latest.LatestNewsFragment">

Następnie dodajmy `SwipeToRefreshLayout` - umożliwia wykonanie akcji pokazujący `ProgressBar`, standardowo wykorzystywany do odświeżania ekranu.

In [ ]:
    <androidx.swiperefreshlayout.widget.SwipeRefreshLayout
        android:id="@+id/swipeToRefreshLayout"
        android:layout_width="match_parent"
        android:layout_height="match_parent">

Kolejne elementy umieścimy w `RelativeLayout`

In [ ]:
        <RelativeLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent">

Dodajmy `RecyclerView` wyświetlający naszą listę wiadomości, `TextView` wyświetlający komunikat błędu oraz przycisk umożliwiający ponowienie pobrania danych z serwera w przypadku braku połączenia.

In [ ]:
        <RelativeLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent">

            <androidx.recyclerview.widget.RecyclerView
                android:id="@+id/latestRecyclerView"
                android:layout_width="match_parent"
                android:layout_height="match_parent"
                android:layout_margin="8dp"/>

            <TextView
                android:id="@+id/errorMessageTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_above="@+id/retryButton"
                android:gravity="center_horizontal"
                android:layout_margin="8dp"
                android:textSize="24sp"
                android:visibility="gone"
                tools:visibility="visible"
                tools:text="Error message"
                />

            <Button
                android:id="@+id/retryButton"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:layout_centerInParent="true"
                android:visibility="gone"
                tools:visibility="visible"
                android:text="@string/po_cz"/>

        </RelativeLayout>

    </androidx.swiperefreshlayout.widget.SwipeRefreshLayout>

</androidx.coordinatorlayout.widget.CoordinatorLayout>

Dodajmy layout dla `FavoriteNewsFragment` - tutaj chcemy wyświetlić `TextView` z informacją o pustej bazie, lub `RecyclerView` jeżeli w bazie istnieje choć jeden element.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.features.favorite.FavoriteNewsFragment">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/favoriteRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="match_parent" />

    <TextView
        android:id="@+id/messageTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_centerInParent="true"
        android:text="@string/brak_wynik_w"
        android:textSize="24sp"
        android:visibility="gone"
        tools:visibility="visible" />

</RelativeLayout>

Dodajmy jeszcze odpowiedni kod obsługujący `BottomNavigation` oraz `Navigation` wraz z `AppBar` do klasy `MainActivity`

In [ ]:
class MainActivity : AppCompatActivity() {

    private val binding: ActivityMainBinding by lazy {
        ActivityMainBinding.inflate(layoutInflater)
    }

    private val navController: NavController by lazy {
        val navHostFragment = supportFragmentManager.findFragmentById(R.id.nav_host_fragment)
                as NavHostFragment
        navHostFragment.findNavController()
    }

    private val appBarConfiguration: AppBarConfiguration by lazy {
        AppBarConfiguration(setOf(R.id.latestNewsFragment, R.id.favoriteNewsFragment))
    }

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(binding.root)


        binding.bottomNavView.setupWithNavController(navController)
        setupActionBarWithNavController(navController, appBarConfiguration)
    }

    override fun onSupportNavigateUp(): Boolean {
        return navController.navigateUp(appBarConfiguration)
                || super.onSupportNavigateUp()
    }
}

### Model danych

Przykładowe żądanie wygląda następująco https://newsdata.io/api/1/news?apikey=YOUR_API_KEY&country=pl&language=pl. W odpowiedzi dostaniemy listę wszystkich wiadomości w formacie `JSON`

In [ ]:
{
   "status":"success",
   "totalResults":71057,
   "results":[
      {
         "title":"Wiceprezes Kruka sprzedał akcje za 4,1 mln zł",
         "link":"https://www.pb.pl/wiceprezes-kruka-sprzedal-akcje-za-41-mln-zl-1140391",
         "keywords":null,
         "creator":null,
         "video_url":null,
         "description":"Michał Zasępa ...
         "content":null,
         "pubDate":"2022-02-04 07:06:55",
         "image_url":null,
         "source_id":"pb",
         "country":[
            "poland"
         ],
         "category":[
            "business"
         ],
         "language":"polish"
      }
      ...
   ],
   "nextPage":1
}

Widzimy że dostajemy listę 71057 elementów, podzielonych na *strony* po 10. W tej aplikacji obsłużymy dane tylko z jednej (losowej) strony i wykonamy prosty *offline caching*. `nextPage` jest wartością którą możemy się posłużyć ładując kolejną porcję 10 wiadomości - do obsługi **paginacji** w aplikacji służy biblioteka `Paging3` (pojawi się w kolejnym projekcie).

Przygotujmy model danych, tym razem wykorzystamy dwa modele danych
- klasy `ArticleResponse` oraz `Article` będą wykorzystywane jako *data transfer object* **dto** - tylko do mapowania odpowiedzi serwera
- klasa `LatestNews` posłuży jako `Entity` w bazie `ROOM`

Jak widzimy powyżej, w odpowiedzi dostajemy metadane i listę artykułów, więc wykorzystamy dwie klasy (pakiet `data.dto`)

In [ ]:
data class ArticleResponse(
    val nextPage: Int,
    val results: List<Article>,
    val status: String,
    val totalResults: Int
)

data class Article(
    val description: String?,
    val image_url: String?,
    val link: String,
    val title: String?,
)

Nie potrzebujemy wszystkich elementów z odpowiedzi, więc część ignorujemy. Na podstawie modelu przygotujme layout  pojedynczego elementu `RecyclerView`

Całość umieścimy w `CardView`

In [ ]:
<androidx.cardview.widget.CardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_marginTop="8dp"
    app:cardBackgroundColor="@color/background"
    app:cardCornerRadius="30dp"
    app:cardElevation="15dp">

Elementy wewnętrzne otoczymy przez `ContraintLayout`

In [ ]:
    <androidx.constraintlayout.widget.ConstraintLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content">

Dodajmy `ImageView` w który wstawimy pobraną grafikę

In [ ]:
        <ImageView
            android:id="@+id/articleImageView"
            android:layout_width="match_parent"
            android:layout_height="250dp"
            android:scaleType="centerCrop"

            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent"
            android:contentDescription="@string/image" />

Dodajemy określoną wysokość i wykorzystujemy właściwość `scaleType="centerCrop"`, który wypełni nam widget `ImageView` pobraną grafiką. Ponieważ na grafice chcemy umieścić tytuł artykułu, musimy się upewnić że będzie on widoczny - w tym celu dodamy gradient do katalogu `drawable`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<shape xmlns:android="http://schemas.android.com/apk/res/android"
    android:shape="rectangle">

    <gradient
        android:angle="90"
        android:startColor="#000"
        android:endColor="#00FFFFFF"/>

</shape>

W layoucie `RecyclerView` dodajmy `ImageView` zawierający wcześniej zdefiniowany gradient i umieśćmy go w tej samej pozycji co poprzedni `ImageView`

In [ ]:
        <ImageView
            android:layout_width="match_parent"
            android:layout_height="150dp"
            android:src="@drawable/gradient"
            app:layout_constraintBottom_toBottomOf="@id/articleImageView"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            android:contentDescription="@string/gradient" />

Kolejnym elementem będzie `ImageVCiew` zawierający ikonę - przez kliknięcie będziemy mogli dodać artykuł do uluibionych

In [ ]:
        <ImageView
            android:id="@+id/favoriteImageView"
            android:layout_width="36dp"
            android:layout_height="36dp"
            android:layout_margin="12dp"
            android:src="@drawable/ic_favorite_unselected"
            app:layout_constraintBottom_toBottomOf="@id/articleImageView"
            app:layout_constraintEnd_toEndOf="parent"
            app:tint="@color/teal_200"
            android:contentDescription="@string/bookmark" />

Cały layout kończymy dwoma polami `TextView` dla tytułu oraz opisu artykułu

In [ ]:
        <TextView
            android:id="@+id/titleTextView"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintEnd_toStartOf="@+id/favoriteImageView"
            app:layout_constraintBottom_toBottomOf="@id/articleImageView"
            android:textSize="16sp"
            android:maxLines="2"
            android:ellipsize="end"
            app:layout_constraintTop_toTopOf="@+id/favoriteImageView"
            tools:text="Rosja zawiesza udział w umowie zbożowej. Jest komentarz Erdogana"
            android:textColor="@color/teal_200"
            android:layout_margin="12dp"/>

        <TextView
            android:id="@+id/descriptionTextView"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:maxLines="3"
            android:ellipsize="end"
            android:layout_marginStart="20dp"
            android:layout_marginEnd="20dp"
            android:layout_marginBottom="12dp"
            android:textColor="@color/teal_200"
            app:layout_constraintTop_toBottomOf="@id/articleImageView"
            tools:text="Notowania ropy naftowej rozpoczęły bieżący tydzień, i jednocześnie ostatnią sesję października, od zniżki. Cena ropy gatunku Brent oscyluje w rejonie 92 USD za baryłkę, a notowania amerykańskiej ropy WTI poruszają się w okolicach 86 USD za baryłkę." />

    </androidx.constraintlayout.widget.ConstraintLayout>
</androidx.cardview.widget.CardView>

Ponieważ nie znamy długości opisu, chcemy go ograniczyć do maksymaslnie trzech linii za pomocą atrybutu `android:maxLines="3"`. Jeżeli opis będzie dłuższy, możemy dodać symbol `...` na końcu za pomocą atrybutu `android:ellipsize="end"`.

### Retrofit

Do pakietu `api` dodajmy interfejs `NwasApi`, będziemy w nim posiadać jedną metodę, zwracającą listę artykułów. Metoda przyjmuje jeden argument - `pageNumber`. Jak wcześniej wspomniałem newsdata.io zwraca listę artykułów podzieloną na *strony* - w tej aplikacji wykorzystamy argument `pageNumber` do pobrania listy 10 artykułów z losowej strony (łatwiej zobaczyć funkcjonalność *offline caching*).

Chcemy artykuły dotyczące Polski w języku polskim

In [ ]:
@GET("news?country=pl&language=pl")

Sama funkcja zwraca obiekt `ArticleResponse`

In [ ]:
suspend fun getLatestNews(
    @Query("page") pageNumber: Int
): ArticleResponse

Musimy jeszcze dodać do żądania nasz klucz, Możemy to zrobić za pomocą adnotacji `@Query`, jednak często dostępny jest `HTTP header`. Z dokumentacji (https://newsdata.io/docs):

`Retrofit` dostarcza adnotację `@Header`, którą możemy tu wykorzystać. Musimy również podać wartość samego klucza, więc do pakietu `util` dodajmy plik `Const`

In [ ]:
const val API_KEY = BuildConfig.NEWS_DATA_IO_KEY

Następnie możemy wykorzystać tą wartość w adnotacji `@Headers` - musimy również podać nazwę nagłówka (znamy z dokumentacji: `X-ACCESS-KEY`)

In [ ]:
interface NewsApi {
    @Headers("X-ACCESS-KEY: $API_KEY")
    @GET("news?country=pl&language=pl")
    suspend fun getLatestNews(
        @Query("page") pageNumber: Int
    ): ArticleResponse
}

Do pakietu `di` dodajmy obiekt `AppModule`

In [ ]:
@Module
@InstallIn(SingletonComponent::class)
object AppModule {

Tym razem napiszemy kilka metod dostarczających poszczególne komponenty wymagane do zwrócenia obiektu o typie `NewsApi`

Rozpocznijmy od dodaania metody dostarczającej iinstancję `LoggingInterceptor`

In [ ]:
    @Provides
    @Singleton
    fun provideLoggingInterceptor(): HttpLoggingInterceptor{
        val interceptor = HttpLoggingInterceptor()
        interceptor.setLevel(HttpLoggingInterceptor.Level.BODY)
        return interceptor
    }

Następnie potrzebujemy `OkHttpClient`

In [ ]:
    @Provides
    @Singleton
    fun provideOkHttpClient(interceptor: HttpLoggingInterceptor): OkHttpClient =
        OkHttpClient.Builder()
            .addInterceptor(interceptor)
            .build()

oraz instancję `Retrofit`

In [ ]:
    @Provides
    @Singleton
    fun provideRetrofit(client: OkHttpClient): Retrofit =
        Retrofit.Builder()
            .baseUrl(BASE_URL)
            .addConverterFactory(GsonConverterFactory.create())
            .client(client)
            .build()

Ostatecznie dodajemy metodę dostarczającą instancję obiektu o typi `NewsApi`

In [ ]:
    @Provides
    @Singleton
    fun provideNewsApi(retrofit: Retrofit): NewsApi = retrofit.create(NewsApi::class.java)

### baza danych `ROOM`

Do pakietu `data.db.entities` dodajmy klasę `NewsArticle`, reprezentującą tabelę w naszej bazie

In [ ]:
@Entity(tableName = "articles")
data class NewsArticle (
    val description: String?,
    val imageUrl: String?,
    @PrimaryKey val url: String,
    val title: String?,
    val isFavorite: Boolean,
    val lastUpdate: Long = System.currentTimeMillis()
)

Zawiera ona dodatkowe pola
- `isFavorite` - czy artykuł został dodany do ulubionych
- `lastUpdate` - przechowujemy czas utworzenia artykułu - wykorzystamy tą informację do czyszczenia bazy ze starych elementów.

Jako klucz główny wykorzystamy adres url artykułu, który jest unikalny.

Nasza aplikacja będzie wykorzystywać zasadę *single source of truth* - innymi słowy, posiadamy dokładnie jedną tabelę (`NewsArticle`), która zawsze posiada zaktualizowane dane.

Tutaj napotykamy problem, ponieważ chceemy również posiadać zapisane ulubione artykuły. W przypadku zastosowania jednej tabeli możemy posłużyć się flagę `isFavorite` do uzyskania listy ulubionych. Jednak problem występuje, gdy chcemy dostać listę najnowszych artykułów - wiemy że jest 10 artykułów, jeżeli użytkownik doda 4 artykuły do ulubionych i następnie odświeży listę, na liście może znaleźć się 10, 11, 12, 13, lub 14 artykułów - innymi słowy możliwa jest sytuacja w której artykuł może znajdować się na liście ulubionych i być w liście najświeższych. Można zastosować *timestamp* podczas dodania i aktualizacji, wtedy listę świeżych artykułów otrzymamy sortując po czasie dodania/aktualizacji i wyciągnięciu pierwszych 10 artykułów - niezależnie od pola `isFavorite`.

Możemy mieć sytuację w której nie jest proste określenie dokładnie które elementy będą przynależeć do której listy. W momencie gdy nie posiadamy dwóch wyróżników (ulubione, najnowsze), tylko cztery lub więcej, napotkamy problem który może nie być łatwy do rozwiązania.

W naszym przypadku, naturalnym rozwiązaniem wydaje się być zastosowanie dwóch niezależnych tabel - jedną dla listy ulubionych oraz drugą dla listy najnowszych - jednak to łamie zasadę *single source of truth*, oraz występuje problem z synchronizacją danych (musimy się upewnić że dane w obu tabelach są aktualne).

W aplikacji zastosujemy drugą tabelę `LatestNews`, będzie ona przechowywała tylko niezbędne informacje - tutaj będzie to wyróżnik artykułu (`url`) (pakiet `data.db.entities`)

In [ ]:
@Entity(tableName = "latest_news")
data class LatestNews (
    val articleUrl: String,
    @PrimaryKey(autoGenerate = true) val id: Int = 0
)

Aby dostać listę najnowszych artykułów wykorzystamy `INNER JOIN` - na podstawie danych zawartych w tabeli `latest_news`, wyciągniemy dane z tabeli `articles`.

Do pakietu `db` dodajmy interfejs `ArticleDao`. Potrzebujemy metodę dodającą listę artykułów do tabeli `articles`.

In [ ]:
@Dao
interface ArticleDao {
    
    @Insert(onConflict = REPLACE)
    suspend fun insertArticles(articles: List<NewsArticle>)

Następnie dodajmy metodę dodającą listę najnowszych artykułów do tabeli `latest_news`

In [ ]:
    @Insert(onConflict = REPLACE)
    suspend fun insertLatestNews(latestNews: List<LatestNews>)

Potrzebujemy również metodę aktualizującą artykuł

In [ ]:
    @Update
    suspend fun updateArticle(newsArticle: NewsArticle)

Ponieważ wykorzystujemy *offline caching*, musimy mieć możliwość usunięcia wszystkich artykułów z tabeli `latest_news`

In [ ]:
    @Query("DELETE FROM latest_news")
    suspend fun delete()

Z tabeli `articles` usuwanie będzie automatycznie, po określonym czasie - tutaj musimyn się upewnić że usuwany artykuł nie jest ulubionym.

In [ ]:
    @Query("DELETE FROM articles WHERE lastUpdate < :timeStampInMillis AND isFavorite = 0")
    suspend fun deleteNotFavoriteOlderThan(timeStampInMillis: Long)

Pozostają dwie metody zwracające listę najnowszych artykułów jako `Flow` oraz listę wszystkich ulubionych artykułów (również `Flow`)

In [ ]:
    @Query("SELECT * FROM latest_news INNER JOIN articles ON articleUrl = url")
    fun getAllLatestNewsArticles(): Flow<List<NewsArticle>>
    
    @Query("SELECT * FROM articles WHERE isFavorite = 1")
    fun getAllFavorite(): Flow<List<NewsArticle>>

Kolejnym elementem będzie utworzenie klasy abstrakcyjnej `ArticleDatabase` (pakiet `db`)

In [ ]:
@Database(entities = [NewsArticle::class, LatestNews::class], version = 1, exportSchema = false)
abstract class ArticlesDatabase : RoomDatabase() {

    abstract fun articleDao(): ArticleDao
}

Do `AppModule` dodajemy metodę dostępową

In [ ]:
    @Provides
    @Singleton
    fun provideArticlesDatabase(app: Application): ArticlesDatabase =
        Room.databaseBuilder(app, ArticlesDatabase::class.java, DATABASE_NAME)
            .build()

### *Dependency Injection*

Dodajmy klasę główną aplikacji w pakiecie głównym

In [ ]:
@HiltAndroidApp
class PolishNewsApplication : Application()

Dodajmy odpowiedni wpis w `AndroidManifest`

In [ ]:
<application
    android:name=".PolishNewsApplication"

`MainActivity` oraz oba fragmenty adnotujemy przez `@AndroidEntryPoint`

In [ ]:
@AndroidEntryPoint
class MainActivity : AppCompatActivity() {

In [ ]:
@AndroidEntryPoint
class LatestNewsFragment : Fragment() {

In [ ]:
@AndroidEntryPoint
class FavoriteNewsFragment : Fragment() {

### `NetworkBoundResource`

Zanim dodamy repozytorium, zaimplementujmy funkcję `networkBoundResource`. Dodajmy klasę `Resources` do pakietu `util`

In [ ]:
sealed class Resource<T>(
    val data: T? = null,
    val throwable: Throwable? = null
) {
    class Success<T>(data: T) : Resource<T>(data)
    class Error<T>(throwable: Throwable, data: T? = null) : Resource<T>(data, throwable)
    class Loading<T>(data: T? = null) : Resource<T>(data)
}

Funkcja `networkBoundResource` będzie podobna do zaimplementowanej w poprzednim przykładzie, pierwsze cztery argumenty pozostają takie same

In [ ]:
inline fun <ResultType, RequestType> networkBoundResource(
    crossinline query: () -> Flow<ResultType>,
    crossinline fetch: suspend () -> RequestType,
    crossinline saveFetchResult: suspend (RequestType) -> Unit,
    crossinline shouldFetch: (ResultType) -> Boolean = {true},

Ponieważ chcemy obsłużyć błędy i pokazać odpowiedni komunikat użytkownikowi, dodamy dwa argumenty - odpowiadają one stanom `Error` oraz `Success`

In [ ]:
    crossinline fetchFailed: (Throwable) -> Unit = {},
    crossinline fetchSuccess: () -> Unit = {}
)

W poprzednim przykładzie tworzyliśmy `Flow`, tym razem wykorzystamy *builder* `ChannelFlow`.

In [ ]:
) = channelFlow {

Gdy chcemy dostać dane z serwera, emitujemy stan `Resource.Loading` zawierający zapisane dane (*cached data*), dzięki temu podczas ładowania możemy użytkownikowi pokazać aktualne dane które mamy zapisane lokalnie. Następnie wykonujemy `fetch` i emitujemy stan (`Resource.Error` lub `Resource.Success`). Po zakończeniu `fetch` funkcja `networkBoundResource` emituje aktualizacje bazy (otrzymane dane).

W naszej aplikacji mamy możliwość dodania artykułu do ulubionych - przez kliknięcie. To może prowadzić do pewnych problemów. Korzystając z `Flow`, stan jest emitowany po zakończeniu całego bloku - innymi słowy musimy dostać odpowiedź z serwera (`Error` lub `Success`) aby móc wyemitować dane - jakiekolwiek zmiany na ui podczas ładowania (`Loading`) będą niedozwolone - więc, jeżeli użytkownik podczas ładowania będzie chciał dodać artykuł do ulubionych, wykorzystując zwykły `Flow`, nie będziemy w stanie zaktualizować ui. Aby taką funkcjonalność móc zapewnić, musimy mieć zapewnione aktualizacje podczas stanu ładowania.

Dzięki zastosowaniu `ChannelFlow` możemy wykonać zadania współbieżnie - możemy otrzymywać aktualizacje podczas stanu ładowania za pomocą drugiej instancji `Flow`. W momencie gdy dostaniemy stan `Success` lub `Error` możemy zakończyć działanie `Flow` wykorzystywanego podczas stanu `Loading`.

W pierwszej kolejności chcemy dostać aktualne dane z bazy i sprawdzić czy chcemy wykonać aktualizację.

In [ ]:
    val data = query().first()
    if (shouldFetch(data)){

Funkcja `query` zwraca `Flow` z danymi z bazy, wywołujemy funkcję `first` ponieważ chcemy pobrać tylko jedne raz dane (pojjedynczą kopię listy) -- innymi słowy, funkcja `first` jest tożsama z funkcją `collect`, lecz po otrzymaniu danych zatrzymuje działanie.

W bloku `if` chcemy wyemitować stan `Loading` (współbieżnie), wykonujemy to przez wykorzystanie `Coroutine`

In [ ]:
        val loading = launch { query().collect{send(Resource.Loading(it))} }

Wartość zwracaną przechowujemy w `loading` aby móc zatrzymać wykonanie. `launch` uruchamia `Coroutine` wewnątrz naszego kanału. Wywołujemy metodę `collect` - zbiera każdą wartość wyemitowaną przez bazę (tutaj dostaniemy zmianę wartości `isFavorite`). Następnie chcemy wysłać dane (wyemitować) wraz ze stanem `Loading` za pomocą metody `send`.

Następnie chcemy pobrać dane z serwera i zapisac w bazie.

In [ ]:
        saveFetchResult(fetch())

Po zapisaniu danych chcemy wykonać metodę `fetchSuccess` i zatrzymać działanie `Coroutine` z `Flow`v emitującym stan `Loading`

In [ ]:
        fetchSuccess()
        loading.cancel()

Następnie chcemy wysłać dane ze stanem `Success`

In [ ]:
        query().collect { send(Resource.Success(it)) }

Jeżeli dostaniemy błąd i pobranie danych z serwera zakończy się niepowodzeniem. Dodajmy blok `try-catch`

In [ ]:
        try {
            saveFetchResult(fetch())
            fetchSuccess()
            loading.cancel()
            query().collect { send(Resource.Success(it)) }
        } catch (t: Throwable) {

W przypadku niepowodzenia chcemy wywołać metodę `fetchFailed` i również zatrzymać działanie `Coroutine` emitującego stan `Loading`

In [ ]:
            fetchFailed(t)
            loading.cancel()

Następnie wysyłamy błąd i to co otrzymamy, wraz ze stanem `Error`

In [ ]:
            query().collect{send(Resource.Error(t, it))}

Jeżeli `shouldFetch` zwróci `false` - czyli nie musimy wykonywać aktualizacji - wysyłamy dane wraz ze stanem `Success`

In [ ]:
    } else query().collect{send(Resource.Success(it))}

Pełny kod metody

In [ ]:
inline fun <ResultType, RequestType> networkBoundResource(
    crossinline query: () -> Flow<ResultType>,
    crossinline fetch: suspend () -> RequestType,
    crossinline saveFetchResult: suspend (RequestType) -> Unit,
    crossinline shouldFetch: (ResultType) -> Boolean = {true},
    crossinline fetchFailed: (Throwable) -> Unit = {},
    crossinline fetchSuccess: () -> Unit = {}
) = channelFlow {
    val data = query().first()
    if (shouldFetch(data)){
        val loading = launch { query().collect{send(Resource.Loading(it))} }
        try {
            saveFetchResult(fetch())
            fetchSuccess()
            loading.cancel()
            query().collect { send(Resource.Success(it)) }
        } catch (t: Throwable) {
            fetchFailed(t)
            loading.cancel()
            query().collect{send(Resource.Error(t, it))}
        }
    } else query().collect{send(Resource.Success(it))}
}

### Repozytorium

Możemy dodać do aplikacji repozytorium - tutaj dodamy logikę odświeżania. Wykorzystamy również wstrzykiwanie przez kosntruktor.

In [ ]:
class NewsRepository @Inject constructor (
    private val api: NewsApi,
    private val db: ArticlesDatabase
) {
    private val dao = db.articleDao()

Dodajmy metodę aktualizującą artykuł w bazie

In [ ]:
suspend fun updateArticle(newsArticle: NewsArticle) = dao.updateArticle(newsArticle)

Następnie potrzebujemy metodę zwracającą strumień wszystkich artykułów

In [ ]:
fun getAllFavoriteArticles(): Flow<List<NewsArticle>> = dao.getAllFavorite()

Oraz metodę usuwającą artykuły, które nie są ulubione - tutaj chcemy usunąć artykuły co określoną ilość czasu.

In [ ]:
suspend fun deleteNonFavoriteArticlesOlderThan(timeStampInMillis: Long) = 
    dao.deleteNotFavoriteOlderThan(timeStampInMillis)

Na koniec dodajmy funkcję `getLatestNews` przyjmującą trzy argumenty
- `requestRefresh` - zmienna `Boolean` określająca czy chcemy odświeżyć 
- `fetchFail`, `fetchSucces` - funkcje, które chcemy zaimplementować z `ViewModel`, więc tutaj tylko je przekazujemy.

W funkcji wywołujemy `networkBoundResource`.

In [ ]:
fun getLatestNews(
    requestRefresh: Boolean,
    fetchFail: (Throwable) -> Unit,
    fetchSuccess: () -> Unit): Flow<Resource<List<NewsArticle>>> = networkBoundResource(

Musimy podać sześć argumentów, pierwszym będzie funkcja zwracająca najnowsze artykuły z bazy.

In [ ]:
    query = {dao.getAllLatestNewsArticles()},

Następnie podajmy funkcję zwracającą listę artykułów z serwera - tutaj posłużymy się argumentem `pageNumber` aby za każdym razem zwrócić listę 10 artykułów z losowej strony.

In [ ]:
    fetch = {
        val response = api.getLatestNews(Random.nextInt(1, 40))
        response.results
    },

Następnym argumentem bedzie funkcja `saveAndFetch`. Funkcja przyjmuje listę arykułów pobranych z serwera. Chcemy umieścić artykuły w bazie, ponieważ mamy dwie tabele, musimy zaktualizować obie.

In [ ]:
    saveFetchResult = { articles ->

Musimy określić które artykuły na świeżo pobranej liście znajdują się już w bazie i są oznaczone jako ulubione. W tym celu pobieramy listę uliubionych artykułów.

In [ ]:
        val favoriteArticles = dao.getAllFavorite().first()

Tworzymy wartość `news` zawierającą artykuły, które chcemy dodać do bazy `articles`

In [ ]:
        val news = articles.map { article ->

Sprawdzamy, czy artykuł na liście znajduje się w `favoriteArticles` i zapisujemy wynik, który później wykorzystamy do utworzenia obiektów `NewsArticle`, które dodamy do bazy `articles`

In [ ]:
            val isFavorite = favoriteArticles.any { it.url == article.link }

Następnie tworzymy obiekt `NewsArticle`, który dodawany jest do listy `news`

In [ ]:
    val news = articles.map { article ->
        val isFavorite = favoriteArticles.any { it.url == article.link }
        NewsArticle(
            title = article.title,
            url = article.link,
            imageUrl = article.image_url,
            isFavorite = isFavorite,
            description = article.description
        )
    }

Dalej, tworzymy listę `latest`, którą dodamy do tabeli `latest_news` - czyli potrzebujemy adresy url ze świeżo utworzonej listy `news`

In [ ]:
    val latest = news.map { article ->
        LatestNews(article.url)
    }

Wykonujemy aktualizację, chcemy się upewnić że wszystkie operacje są wykonane, więc wykorzystujemy `withTransaction`

In [ ]:
    db.withTransaction {
        dao.apply {
            delete()
            insertArticles(news)
            insertLatestNews(latest)
        }
    }

Pełna funkcja `saveFetchResult`

In [ ]:
    saveFetchResult = { articles ->
        val favoriteArticles = dao.getAllFavorite().first()
        val news = articles.map { article ->
            val isFavorite = favoriteArticles.any { it.url == article.link }
            NewsArticle(
                title = article.title,
                url = article.link,
                imageUrl = article.image_url,
                isFavorite = isFavorite,
                description = article.description
            )
        }

        val latest = news.map { article ->
            LatestNews(article.url)
        }

        db.withTransaction {
            dao.apply {
                delete()
                insertArticles(news)
                insertLatestNews(latest)
            }
        }
    },

Jako `fetchSuccess` przekazujemy `fetchSuccess`, którą dostajemy jako argument metody `getLatestNews`

In [ ]:
    fetchSuccess = fetchSuccess,

W implementacji `fetchFail` chcemy rzucić wyjątek, którego nie chcemy obsługiwać, w pozostałych przypadkach wykonujemy metodę `fetchFail`, którą dostajemy jako argument `getLatestNews`

In [ ]:
    fetchFailed = { t ->
        if (t !is HttpException && t !is IOException)
            throw t
        fetchFail(t)
    },

Ostatnim argumentem jest funkcja `shouldFetch` - tutaj zaimplementujemy logikę odświeżania listy. Chcemy odświeżyć jeżeli `requestRefresh`, który dostajemy w argumencie metody ma wartość `true`

In [ ]:
    shouldFetch = { cached ->
        if (requestRefresh) true else {

W przeciwnym razie, chcemy sprawdzić czy minął określony czas. Chcemy odświeżać co 10 minut, więc dodajmy odpowiednią wartość do pliku `Const`

In [ ]:
val TIME_TO_REFRESH_DATA = TimeUnit.MINUTES.toMillis(10)

Sprawdzamy czy różnica między najmniejszą wartością `lastUpdate` z danych w bazie a aktualnym czasem przekracza 10 minut i zwracamy wartość `Boolean`

In [ ]:
    shouldFetch = { cached ->
        if (requestRefresh) true else {
        val oldestTimeStamp = cached.minByOrNull { article -> article.lastUpdate }?.lastUpdate
        oldestTimeStamp == null || oldestTimeStamp < System.currentTimeMillis() - TIME_TO_REFRESH_DATA
        }
    }

### LatestNewsViewModel

Przejdźmy do pakietu `ui.features.latest` i dodajmy `LatestViewModel`

In [ ]:
@HiltViewModel
class LatestNewsViewModel @Inject constructor(
    private val repository: NewsRepository
) : ViewModel()

Dodajmy kod pozwalający pobrać dane z serwera. 

In [ ]:
val latestNews = repository.getLatestNews()

Tutaj napotykamy problem, `Flow` który otrzymujemy z funkcji `networkBoundRresource` jest *zimny*, czyli cały blok kodu w funkcji `networkBoundResource` 

```kotlin
// NetworkBoundResource
 = channelFlow {
    val data = query().first()
    if (shouldFetch(data)){
        val loading = launch { query().collect{send(Resource.Loading(it))} }
        try {
            saveFetchResult(fetch())
            fetchSuccess()
            loading.cancel()
            query().collect { send(Resource.Success(it)) }
        } catch (t: Throwable) {
            fetchFailed(t)
            loading.cancel()
            query().collect{send(Resource.Error(t, it))}
        }
    } else query().collect{send(Resource.Success(it))}
}
```


będzie wykonany za każdym razem gdy wywołamy metodę `getLatestNews`. Za każdym razem gdy zmienimy odświeżymy fragment (przejdziemy na inny fragment przez nawigację i z powrotem, lub zmienimy orientację urządzenia) metoda `getLatestNews` będzie wywołana, która rozpocznie działanie od początku. Aby tego uniknąć wykoszystamy `StateFlow` - przechowuje swoją ostatnią wartość, więc nie ma konieczności wykonania całego bloku `channelFlow` z funkcji `networkBoundResource`.

`StateFlow` możemy utworzyć przez wywołanie metody `stateIn`, przyjmuje ona trzy parametry
- `CoroutineScope` - zakres w którym `Flow` będzie zebrany, tutaj wykorzystamy `viewModelScope` co pozwoli na zachowanie stanu podczas zmian cyklu życia fragmentu. Gdy `ViewModel` na zakresie którego `StateFlow` będzie wykorzystany, zostanie zniszczony, sam `StateFlow` zostanie zatrzymany
- `SharingStarted` - dzięki temu parametrowi możemy określić, kiedy nasz `StateFlow` staje się aktywny. Tutaj wykorzystamy `SharingStarted.Lazily` - oznacza to, że ten `StateFlow` będzie aktywny dopiero gdy jakiś obiekt rozpocznie zbieranie `latestNews`
- `initialValue` - wartość początkowa, tutaj przekazujemy `null`

In [ ]:
val latestNews = repository.getLatestNews().stateIn(viewModelScope, SharingStarted.Lazily, null)

Kolejnym elementem będzie implementacja metody umożliwiwającej odświeżenie danych z serwera na żądanie - będziemy ją wykorzystywać w odpowiedzi na zdarzenie `swipeToRefresh`.

In [ ]:
fun refreshOnDemand() {}

Tutaj napotykamy kolejny problem - w jaki sposób możemy ponownie wywołać metodę `getLatestNews` i podłączyć otrzymany `Flow` do zdefiniowanej wcześniej wartości `latestNews`. Niestety nie jest to proste zadanie, ponieważ we fragmencie wykonywana będzie metoda `collect` na poprzedniej wersji `Flow`. Czyli nie mogę zrobić

In [ ]:
// tak nie można

// dostajemy Flow "pierwszy"
var latestNews = repository.getLatestNews().stateIn(viewModelScope, SharingStarted.Lazily, null)
fun refreshOnDemand() {
    // inna instancja Flow "drugi"
    latestNews = repository.getLatestNews().stateIn(viewModelScope, SharingStarted.Lazily, null)
}

// Fragment
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    viewLifecycleOwner.lifecycleScope.launchWhenStarted {
                viewModel.latestNews.collect{} // zawsze zbieramy z "pierwszego" flow
}

Aby rozwiązać ten problem, wykorzystamy inny obiekt `Flow`, który będzie działał jako *trigger* - gdy wyemituje wartość, nasz `latestNews` `Flow` przełączy się na inny `Flow`.

W pierwszej kolejności potrzebujemy `Flow`, który będzie działał jako sygnalista zmian - w tym celu wykorzystujemy *kanały* (`Channel`) - metoda komunikacji pomiędzy dwiema `Coroutines` - `coroutine1` może coś umieścić w kanale, `coroutine2` może odebrać z kanału. Ponieważ sam kanał również dziąła na `Coroutine`, obie strony mogą zawiesić wykonanie w oczekiwaniu na drugą stronę.

Wykorzystamy kanała aby umieścić w nim sygnał do przełączenia się na inny `Flow` - chwilowo wykorzystamy typ `Unit`

In [ ]:
private val refreshChannelTrigger = Channel<Unit>()

Do kanału możemy dostarczyć wartości tylko z wewnątrz `Coroutine`

In [ ]:
fun refreshOnDemand() {
        viewModelScope.launch { refreshChannelTrigger.send(Unit) }
}

Na naszym `refreshChannelTrigger` wykonujemy metodę `send` i dostarczamy wartość o typie `Unit` do kanału, sygnalizując konieczność zmiany.

Następnie tworzymy następną wartość, w której odbieramy wartość z `refreshChannelTrigger` jak `Flow`

In [ ]:
private val refreshTrigger = refreshChannelTrigger.receiveAsFlow()

Teraz możemy dostać sygnały do zmiany przez strumień `refreshTrigger` - jest to *ciepłe* lub *gorące* źródło danych - produkuje wartości niezależnie od tego czy jest zbieranee czy nie.

Teraz dodajmy funkcjonalność do wartości `latestNews`

In [ ]:
val latestNews = refreshTrigger.flatMapLatest {
    repository.getLatestNews()
}.stateIn(viewModelScope, SharingStarted.Lazily, null)

Za każdym razem gdy pojawi się wartość o typie `Unit` (czyli zostanie zasygnalizowana zmiana), zostanie wywołany blok `flatMapLatest` i nastąpi przełączenie na `Flow`, który otrzymamy przy wywołaniu `repository.getLatestNews` (poprzedni `Flow` jestb przerywany).

Gdybyśmy tutaj wykorzystali `map`, dostalibyśmy `StateFlow<Flow<Resource<List<NewsArticle>>>>`, potrzebujemy `Flow` - dlatego wywołujemy `flatMap` aby otrzymać `Flow<Resource<List<NewsArticle>>>`. Ponieważ za każdym przełączeniem chcemy przerwać działanie poprzedniego `Flow`, wywołujemy `flatMapLatest` 

Jeżeli już jesteśmy w stanie `Loading`, chcemy zablokować możliwość ponownego odświeżenia.

In [ ]:
fun refreshOnDemand() {
    if (latestNews.value !is Resource.Loading)
        viewModelScope.launch { refreshChannelTrigger.send(Unit) }
}

Potrzebujemy jeszcze jedną metodę - gdy nasz fragment staje się aktywny chcemy wykonać odświeżenie

In [ ]:
fun refreshOnStart() {
    if (latestNews.value !is Resource.Loading)
        viewModelScope.launch { refreshChannelTrigger.send(Unit) }
}

Wróćmy do `latestNews`

In [ ]:
val latestNews = refreshTrigger.flatMapLatest {
    repository.getLatestNews()
}.stateIn(viewModelScope, SharingStarted.Lazily, null)

`repository.getLatestNews()` wymagając trzech argumentów w celu pokazania odpowiednich błędów i stanu. Jeżeli wystąpi błąd z połączeniem, pokażemy `SnackBar` z kodem błędu. W pierwszej kolejności dodajmy funkcję rozszerzającą do pliku `Util` (pakiet `util`)

In [ ]:
fun Fragment.showSnackbar(
    message: String,
    duration: Int = Snackbar.LENGTH_LONG,
    view: View = requireView()
){
    Snackbar.make(view, message, duration).show()
}

W naszej metodzie `getLatestNews`, jako argument `onFetchFail` chcemy pokazać `SnackBar` z odpowiednim komunikatem.

In [ ]:
repository.getLatestNews(
    fetchFail = { t ->
)

Tutaj znów napotykamy kłopot - `ViewModel` nie może posiadać referencji do fragmentu, lecz w jakiś sposób musimy wywołać metodę `showSnakcbar`. Zamiast wywoływać metodę wprost, wyemitujemy obserwowalne zdarzenie (`event`) - w tym celu wykorzystamy kanały.

In [ ]:
private val eventChannel = Channel<>()

Jako typ utworzymy klasę domkniętą wewnątrz `ViewModel` - analogicznie do klasy `Resources`

In [ ]:
sealed class Event {
    data class ShowErrorMessage(val t: Throwable) : Event()
}

Teraz jako typ możemy wykorzystać `Event`

In [ ]:
private val eventChannel = Channel<Event>()

Podobnie jak poprzednio, chcemy posiadać `Flow`, przez który możemy wysłać odpowiednie zdarzenia do fragmentu.

In [ ]:
val events = eventChannel.receiveAsFlow()

Zaimplementujmy `onFetchFail` - mnusimy wykorzystać `Coroutines` do przekazania wartości do kanału.

In [ ]:
val latestNews = refreshTrigger.flatMapLatest {
    repository.getLatestNews(
        fetchFail = { t ->
            viewModelScope.launch { eventChannel.send(Event.ShowErrorMessage(t)) }
        },     
    )
}.stateIn(viewModelScope, SharingStarted.Lazily, null)

Przy wykonaniu odświeżania danych chcemy również rozpocząć przeglądanie nowych danych od pierwszego elementu, w tym celu musimy przestawić nasz `RecyclerView` na pierwszą pozycję. Ale jednocześnie chcemy uniknąć przewijania do pierwszego elementu przy zmianie i powrocie do fragmentu, lub przy zmianie orientacji urządzenia. W tym celu zdefiniujmy flagę.

In [ ]:
var pendingScrollToTop = false

Przestawmy ją przy wywołaniu `getLatestNews`

In [ ]:
val latestNews = refreshTrigger.flatMapLatest {
    repository.getLatestNews(
        fetchFail = { t ->
            viewModelScope.launch { eventChannel.send(Event.ShowErrorMessage(t)) }
        },
        fetchSuccess = { pendingScrollToTop = true }
    )
}.stateIn(viewModelScope, SharingStarted.Lazily, null)

Ostatnim elementem pozostaje logika odświeżania. W repozytorium zimplementowaliśmy `shouldFetch`

```kotlin
shouldFetch = { cached ->
    if (requestRefresh) true else {
    val oldestTimeStamp = cached.minByOrNull { article -> article.lastUpdate }?.lastUpdate
    oldestTimeStamp == null || oldestTimeStamp < System.currentTimeMillis() - TIME_TO_REFRESH_DATA
    }
}
```

Jeżeli `requestRefresh` ma wartość `true`, powinniśmy odświeżyć dane, w przeciwnym razie posługujemy się czasem aby określić czy chcemy odświeżyć. `requestRefresh` jest ostatnim argumentem, który musimy przekazać do funkcji `getLatestNews` w naszym `ViewModel`. Mamy dwa możliwe stany
- normalny trry odświeżenia - związany z upłynięcie czasu
- odświeżenie na żądanie - przykładowo przy `swipeToRefresh`

Aby opisać te dwa stany, wykorzystajmy `enum class`, który umieścimy wewnątrz `ViewModel`

In [ ]:
enum class Refresh {
    REQUEST, NORMAL
}

Zmodyfikujmy nasze metody `refreshOnDemand` oraz `refreshOnStart`

In [ ]:
private val refreshChannelTrigger = Channel<Refresh>()

...

fun refreshOnDemand() {
    if (latestNews.value !is Resource.Loading)
        viewModelScope.launch { refreshChannelTrigger.send(Refresh.REQUEST) }
}

fun refreshOnStart() {
    if (latestNews.value !is Resource.Loading)
        viewModelScope.launch { refreshChannelTrigger.send(Refresh.NORMAL) }
}

Dodajmy ostatni argument do `getLatestNews`. Ponieważ zmieniliśmy typ naszego sygnalizatora z `Unit` na `Refresh`, `flatMapLatest` dostarczy nam najnowszą wartość - stąd dowiemy się w jakim trybiee chcemy odświeżyć dane

In [ ]:
val latestNews = refreshTrigger.flatMapLatest { refresh ->

Pierwszym argumentem `getLatestNews` jest `requestRefresh`, w trybie nomalnym chcemy przekazać `flase`, więc możemy zapisać

In [ ]:
refresh == Refresh.REQUEST,

Ostatnia metod umiżliwia dodanie artykułu do ulubionych

In [ ]:
fun addFavorite(newsArticle: NewsArticle){
    val favorite = newsArticle.isFavorite
    val updatedArticle = newsArticle.copy(isFavorite = !favorite) // tworzę kopię artykułu
    viewModelScope.launch { repository.updateArticle(updatedArticle) }
}

Gdy wykorzystujemy reaktywne źródła danych (`LiveData`, `Flow`), powinniśmy wykorzystywać tylko **niemutowalne** właściwości, stąd zastosowanie kopiowania.

Pełny kod `LatestViewModel`

In [ ]:
@HiltViewModel
class LatestNewsViewModel @Inject constructor(private val repository: NewsRepository) : ViewModel() {

    private val eventChannel = Channel<Event>()
    val events = eventChannel.receiveAsFlow()

    private val refreshChannelTrigger = Channel<Refresh>()
    private val refreshTrigger = refreshChannelTrigger.receiveAsFlow()

    var pendingScrollToTop = false

    @OptIn(ExperimentalCoroutinesApi::class)
    val latestNews = refreshTrigger.flatMapLatest { refresh ->
        repository.getLatestNews(
            refresh == Refresh.REQUEST,
            fetchFail = { t ->
                viewModelScope.launch { eventChannel.send(Event.ShowErrorMessage(t)) }
            },
            fetchSuccess = { pendingScrollToTop = true }
        )
    }.stateIn(viewModelScope, SharingStarted.Lazily, null)

    init {
        viewModelScope.launch {
            repository.deleteNonFavoriteArticlesOlderThan(
                System.currentTimeMillis() - TIME_TO_DELETE_NOT_FAVORITE_ARTICLES
            )
        }
    }


    fun refreshOnDemand() {
        if (latestNews.value !is Resource.Loading)
            viewModelScope.launch { refreshChannelTrigger.send(Refresh.REQUEST) }
    }

    fun refreshOnStart() {
        if (latestNews.value !is Resource.Loading)
            viewModelScope.launch { refreshChannelTrigger.send(Refresh.NORMAL) }
    }
    
    fun addFavorite(newsArticle: NewsArticle){
        val favorite = newsArticle.isFavorite
        val updatedArticle = newsArticle.copy(isFavorite = !favorite)
        viewModelScope.launch { repository.updateArticle(updatedArticle) }
    }

    enum class Refresh {
        REQUEST, NORMAL
    }

    sealed class Event {
        data class ShowErrorMessage(val t: Throwable) : Event()
    }
}

### FavoriteViewModel

Przejdźmy do `FavoriteViewModel`, potrzebujemy dostać wszystkie ulubione artykuły, więc jak poprzednio zastosujemy `stateIn`

In [ ]:
@HiltViewModel
class FavoriteViewModel @Inject constructor (
    private val repository: NewsRepository
) : ViewModel() {
    val favorites = repository.getAllFavoriteArticles()
        .stateIn(viewModelScope, SharingStarted.Lazily, null)
}

Drugą i ostatnią funkcją, której wymagamy, to możliwość dodania/usunięcia artykułu do bazy

In [ ]:
@HiltViewModel
class FavoriteViewModel @Inject constructor (
    private val repository: NewsRepository
) : ViewModel() {
    val favorites = repository.getAllFavoriteArticles()
        .stateIn(viewModelScope, SharingStarted.Lazily, null)

    fun addFavorite(newsArticle: NewsArticle){
        val favorite = newsArticle.isFavorite
        val updatedArticle = newsArticle.copy(isFavorite = !favorite)
        viewModelScope.launch { repository.updateArticle(updatedArticle) }
    }
}

### RecyclerView

Będziemy wyświetlać nasze listy za pomocą `RecyclerView`, więc jak zwykle musimy utworzyć adapter, viewholder oraz comparator. Zrobimy to w pakiecie `shared` - obie listy `RecyclerView` będą wykorzystywać te same elementy.

In [ ]:
class ArticleComparator : DiffUtil.ItemCallback<NewsArticle>() {
    override fun areItemsTheSame(oldItem: NewsArticle, newItem: NewsArticle): Boolean = 
    newItem.url == oldItem.url
    override fun areContentsTheSame(oldItem: NewsArticle, newItem: NewsArticle): Boolean = 
    newItem == oldItem
}

Przejdźmy do `ViewHolder`, chcemy dodać `onClickListener` dla całego elementu oraz dla `ImageView`, przez który będziemy dodawać/usuwać element z listy ulubionych. Tym razem przekażemy obie funkcje jako parametry konstruktora. Nie chcemy tworzyć obiektów nasłuchiwaczy w funkcji `bind`, ponieważ jest ona wywoływana często, więc prowadzi to do lawinowego tworzenia obiektów.

In [ ]:
class ArticleViewHolder(
    private val binding: ItemArticleRvBinding,
    private val onItemClick: (Int) -> Unit,
    private val onFavoriteClick: (Int) -> Unit
    )
    : RecyclerView.ViewHolder(binding.root){

Nasze dwie funkcje wywołujemy w bloku `init`, przy tworzeniu samego `ViewHolder`

In [ ]:
    init {
        binding.apply {
            root.setOnClickListener { click(onItemClick) }
            favoriteImageView.setOnClickListener { click(onFavoriteClick) }
        }
    }
    
    private fun click(click: (Int) -> Unit) {
        val position = bindingAdapterPosition
        if (position != RecyclerView.NO_POSITION) click(position)
    }

Funkcja `click` przyjmuje funkcję jako parametr. Pozycje elementu dostajemy wywołując metodę `bindingAdapterPosition`, sprawdzamy czy dostajemy prawidłową pozycję i wywołujemy funkcję `click`.

Dalej dodajemy funkcję `bind`

In [ ]:
fun bind(item: NewsArticle){
    binding.apply {

Ustawiamy tekst tytułu oraz opisu, pamiętając że są to elementy zerowalne

In [ ]:
    titleTextView.text = item.title?:""
    descriptionTextView.text = item.description?:""

Dodajemy grafiki za pomocą biblioteki `Glide`

In [ ]:
    if (item.imageUrl != null)
        Glide.with(itemView)
            .load(item.imageUrl)
            .into(articleImageView)

Ponieważ `imageUrl` może mieć wartość `null` dodajemy blok `else` w którym czyścimy bufor (`Glide` zachowuje elementy, więc możemy duplikować grafiki gdy dostaniemy `null`) i ustawiamy naszą grafikę `no_image`

In [ ]:
    else {
        Glide.with(itemView).clear(articleImageView)
        articleImageView.setImageResource(R.drawable.no_image)
    }

Dodajmy również obsługę `ImageView` reprezentującym informację o tym, czy artykuł znajduje się w ulubionych.

In [ ]:
    favoriteImageView.setImageResource(
        when{
            item.isFavorite -> R.drawable.ic_favorite_selected
            else -> R.drawable.ic_favorite_unselected
        }
    )

### LatestNewsFragment

Rozpoczynamy od dodania odpowiednich pól

In [ ]:
private val viewModel: LatestNewsViewModel by viewModels()

private val menuHost: MenuHost by lazy { requireActivity() }

lateinit var binding: FragmentLatestNewsBinding

Chcemy dodać opcję odświeeżenia listy w `menu` na `ActionBar`, w tym celu musimy wykorzystać `MenuHost`. W metodzie `onCreateView` ustawiamy layout.

In [ ]:
override fun onCreateView(
    inflater: LayoutInflater, container: ViewGroup?,
    savedInstanceState: Bundle?
): View {
    binding = FragmentLatestNewsBinding.inflate(layoutInflater, container, false)
    return binding.root
}

Dodajmy funkcję obsługującą `menu`

In [ ]:
private fun handleMenu() {
    menuHost.addMenuProvider(object : MenuProvider {
        override fun onCreateMenu(menu: Menu, menuInflater: MenuInflater) {
            menuInflater.inflate(R.menu.menu_latest, menu)
        }

        override fun onMenuItemSelected(menuItem: MenuItem): Boolean {
            return when (menuItem.itemId) {
                R.id.refresh -> {
                    viewModel.refreshOnDemand()
                    true
                }
                else -> false
            }
        }
    }, viewLifecycleOwner, Lifecycle.State.RESUMED)
}

Korzystamy z implementacji `MenuProvider`, w metodzie `onCreateMenu` przekazujemy layout oraz samo `menu`. W metodzie `onMenuItemSelected` dodajemy obsługę wszystkich pozycji. Tutaj chcemy wykonać `refreshOnDemand`.

Chcemy również nadpisać funkcję `onStart` i wywołać `refreshOnStart`

In [ ]:
override fun onStart() {
    super.onStart()
    viewModel.refreshOnStart()
}

Napiszmy również funkcję zmieniającą nam stan layoutu (widoczność elementów w zależności od stanu)

In [ ]:
private fun FragmentLatestNewsBinding.layoutState(result: Resource<List<NewsArticle>>) {
    swipeToRefreshLayout.isRefreshing = result is Resource.Loading
    latestRecyclerView.isVisible = !result.data.isNullOrEmpty()
    errorMessageTextView.isVisible = result.throwable != null && result.data.isNullOrEmpty()
    retryButton.isVisible = result.throwable != null && result.data.isNullOrEmpty()
    errorMessageTextView.text =
        getString(R.string.blad, result.throwable?.localizedMessage ?: R.string.nieznany_blad)
}

Przejdźmy do metody `onViewCreated` i rozpocznijmy od dodania adaptera

In [ ]:
val articleAdapter = ArticleAdapter(
    onItemClick = {
        val uri = Uri.parse(it.url)
        val intent = Intent(Intent.ACTION_VIEW, uri)
        requireActivity().startActivity(intent)
    },
    onFavoriteClick = { viewModel.addFavorite(it) }
)

Musimy dostarczyć implementacje dwóch funkcji jako argrumenty, dla kliknięć w sam element oraz w ikonę ulubionych. Po kliknięciu w element wykorzystamy `Implicit Intent` aby otworzyć stronę zawierającą cały artykuł. Po naciśnięciu w ikonę ulubionych wykonujemy metodę `addFavorite`.

Dodajmy `RecyclerView`

In [ ]:
binding.apply {
    latestRecyclerView.apply {
        adapter = articleAdapter
        layoutManager = LinearLayoutManager(requireContext())
        itemAnimator?.changeDuration = 0
    }

Ustawiamy `itemAnimator.Dutration` na 0, aby pozbyć się efektu *flashu* przy przeładowaniu danych.

Następnie chcemy rozpocząć zbieranie danych.

In [ ]:
viewLifecycleOwner.lifecycleScope.launchWhenStarted {
                viewModel.latestNews.collect{

Wykorzystujemy `launchWhenStarted` aby się upewnić, że fragment jest co najmniej w stanie `START` zanim rozpoczniemy zbieranie i obsługę danych. Sprawdzamy czy dostajemy `Resource<List<NewsArticle>>`, jeżeli nie - wychodzimy z bloku i kończymy działanie

In [ ]:
        val result = it ?: return@collect

Dalej wywołujemy wcześniej zdefiniowany `layoutState`

In [ ]:
        layoutState(result)

Nastęnie wykonujemy `submitList` na naszym `RecyclerView`, dostarczając nową wersję danych. Tutaj chcemy też przewinąć do pierwszej pozycji, jeżeli `pendingScrollToTop` ma wartość `true`.

In [ ]:
        articleAdapter.submitList(result.data){
            if (viewModel.pendingScrollToTop) {
                latestRecyclerView.scrollToPosition(0)
                viewModel.pendingScrollToTop = false
            }
        }

Pełny kod

In [ ]:
viewLifecycleOwner.lifecycleScope.launchWhenStarted {
    viewModel.latestNews.collect{
        val result = it ?: return@collect
        layoutState(result)
        articleAdapter.submitList(result.data){
            if (viewModel.pendingScrollToTop) {
                latestRecyclerView.scrollToPosition(0)
                viewModel.pendingScrollToTop = false
            }
        }
    }
}

Następnie ustawmy nasłuchiwacze dla `RefreshListener` oraz `retryButton`

In [ ]:
swipeToRefreshLayout.setOnRefreshListener { viewModel.refreshOnDemand() }
retryButton.setOnClickListener { viewModel.refreshOnDemand() }

Ostatnim elementem jest pokazanie komunikatu o błędzie

In [ ]:
viewLifecycleOwner.lifecycleScope.launchWhenStarted {
    viewModel.events.collect{event -> // zbieramy zdarzenia (tylko jedno w aplikacji)
        when(event){
            is LatestNewsViewModel.Event.ShowErrorMessage ->
                showSnackbar(getString(R.string.blad, event.t.localizedMessage?: R.string.nieznany_blad))
        }.exhaustive
    }
}

Ponieważ nie chcemy obsługiwać ogólnego błędu (a więc stosować wyrażenia `else`), możemy na końcu dodać `exhaustive`, który oznacza że wszystkie opcje są wyczerpane. Samą implementację dodajemy do pliku `Util`

In [ ]:
val <T> T.exhaustive: T
    get() = this

Pełny kod:

In [ ]:
@AndroidEntryPoint
class LatestNewsFragment : Fragment() {

    private val viewModel: LatestNewsViewModel by viewModels()

    private val menuHost: MenuHost by lazy { requireActivity() }

    lateinit var binding: FragmentLatestNewsBinding

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentLatestNewsBinding.inflate(layoutInflater, container, false)
        return binding.root
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)

        val articleAdapter = ArticleAdapter(
            onItemClick = {
                val uri = Uri.parse(it.url)
                val intent = Intent(Intent.ACTION_VIEW, uri)
                requireActivity().startActivity(intent)
            },
            onFavoriteClick = { viewModel.addFavorite(it) }
        )
        binding.apply {
            latestRecyclerView.apply {
                adapter = articleAdapter
                layoutManager = LinearLayoutManager(requireContext())
                itemAnimator?.changeDuration = 0
            }

            viewLifecycleOwner.lifecycleScope.launchWhenStarted {
                viewModel.latestNews.collect{
                    val result = it ?: return@collect
                    layoutState(result)
                    articleAdapter.submitList(result.data){
                        if (viewModel.pendingScrollToTop) {
                            latestRecyclerView.scrollToPosition(0)
                            viewModel.pendingScrollToTop = false
                        }
                    }
                }
            }

            swipeToRefreshLayout.setOnRefreshListener { viewModel.refreshOnDemand() }
            retryButton.setOnClickListener { viewModel.refreshOnDemand() }
            viewLifecycleOwner.lifecycleScope.launchWhenStarted {
                viewModel.events.collect{event ->
                    when(event){
                        is LatestNewsViewModel.Event.ShowErrorMessage ->
                            showSnackbar(getString(
                                R.string.blad, 
                                event.t.localizedMessage?: R.string.nieznany_blad))
                    }.exhaustive
                }
            }
        }

        handleMenu()
    }

    private fun handleMenu() {
        menuHost.addMenuProvider(object : MenuProvider {
            override fun onCreateMenu(menu: Menu, menuInflater: MenuInflater) {
                menuInflater.inflate(R.menu.menu_latest, menu)
            }

            override fun onMenuItemSelected(menuItem: MenuItem): Boolean {
                return when (menuItem.itemId) {
                    R.id.refresh -> {
                        viewModel.refreshOnDemand()
                        true
                    }
                    else -> false
                }
            }
        }, viewLifecycleOwner, Lifecycle.State.RESUMED)
    }

    private fun FragmentLatestNewsBinding.layoutState(result: Resource<List<NewsArticle>>) {
        swipeToRefreshLayout.isRefreshing = result is Resource.Loading
        latestRecyclerView.isVisible = !result.data.isNullOrEmpty()
        errorMessageTextView.isVisible = result.throwable != null && result.data.isNullOrEmpty()
        retryButton.isVisible = result.throwable != null && result.data.isNullOrEmpty()
        errorMessageTextView.text =
            getString(R.string.blad, result.throwable?.localizedMessage ?: R.string.nieznany_blad)
    }

    override fun onStart() {
        super.onStart()
        viewModel.refreshOnStart()
    }
}

### FavoriteNewsFragment

Tutaj nie ma żadnych nowych elementów, więc tylko pełny kod.

In [ ]:
@AndroidEntryPoint
class FavoriteNewsFragment : Fragment() {

    lateinit var binding: FragmentFavoriteNewsBinding

    private val viewModel: FavoriteViewModel by viewModels()

    override fun onCreateView(
        inflater: LayoutInflater,
        container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentFavoriteNewsBinding.inflate(layoutInflater, container, false)
        return binding.root
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)

        val articleAdapter = ArticleAdapter(
            onItemClick = {
                val uri = Uri.parse(it.url)
                val intent = Intent(Intent.ACTION_VIEW, uri)
                requireActivity().startActivity(intent)
            },
            onFavoriteClick = { viewModel.addFavorite(it) }
        )

        binding.apply {
            favoriteRecyclerView.apply {
                adapter = articleAdapter
                layoutManager = LinearLayoutManager(requireContext())
            }

            viewLifecycleOwner.lifecycleScope.launchWhenStarted {
                viewModel.favorites.collect{
                    val favorites = it ?: return@collect
                    articleAdapter.submitList(favorites)
                    messageTextView.isVisible = favorites.isEmpty()
                    favoriteRecyclerView.isVisible = favorites.isNotEmpty()
                }
            }
        }
    }
}

Możemy przetestować aplikację

<table><tr><td><img src="https://media3.giphy.com/media/uER0JwmDholYnLXChH/giphy.gif?cid=790b76119058b2b2f72a5a208fa580c5adabc5d007d55022&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media2.giphy.com/media/GprR3qLfjPdna6GY4m/giphy.gif?cid=790b76116f13b2b93dea78771b7ec3023cdf6f9877e4443a&rid=giphy.gif&ct=g" width="200" /></td>
<td><img src="https://media2.giphy.com/media/GmKwtKs1YnfnrQxEPG/giphy.gif?cid=790b761102b60ac194f6443efdb6ac545a2d184ce159195b&rid=giphy.gif&ct=g" width="200" /></td></tr></table>